# Customer Segmentation - Complete

This notebook aims to analyse the purchases made by approximately 4,000 customers from a UK based online E-retail during the period of one year (from 01/12/2010 to 09/12/2011).

**1. Loading and preparing the original data**

**2. Recency, Frequency, and Monetary Value analysis**

**3. Data pre-processing for clustering**

**4. Customer Segmentation with K-means**

**5. Finding growth opportunities**

## 0. Preliminary Settings

Before starting any analysis, we make sure that the basic and general-use libraries (numpy, pandas, etc.) are correctly imported.

We import the libraries including the Matplotlib-based statistical data visualization package "seaborn" to make graphs from our data.

In [1]:
# data upload
import pandas as pd
import io
# data manipulation and visualization
import matplotlib.pyplot as plt
import numpy as np
import itertools
import seaborn as sns
import datetime
import plotly.graph_objs as go
from plotly.offline import iplot
import nltk
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import matplotlib as mpl
from matplotlib import cm
from wordcloud import WordCloud
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import ensemble
from sklearn.model_selection import learning_curve
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn import neighbors
from sklearn.svm import LinearSVC
import datetime as dt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mohamed.a.eshra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mohamed.a.eshra\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## 1. Loading and preparing the data

We load the data into our environment.

In [ ]:
xls="https://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx"
sheet1 = pd.read_excel(xls, sheet_name='Year 2009-2010')
sheet2 = pd.read_excel(xls, sheet_name='Year 2010-2011')

# We append the two DataFrames
online = sheet1.append(sheet2, ignore_index=True)

# And we convert the data type of the InvoiceDate column to datetime
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'])

# And we rename several columns
online.columns = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

We confirm that the dataset is appearing correctly.

In [ ]:
online.head()

In [ ]:
online.info()

And it's always a good practice to review data types.
As you can see, most of the columns are strings marked as objects.

Next, we review the data types, the number of null values and their percentage with respect to the total number of entries.

In [ ]:
# We reveiew the data types and the number and percentage of null values
tab_info= pd.DataFrame(online.dtypes).T.rename(index={0:'column type'})
tab_info= tab_info.append(pd.DataFrame(online.isnull().sum()).T.rename(index={0:'number of null values'}))
tab_info= tab_info.append(pd.DataFrame(online.isnull().sum()/online.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
tab_info

When looking at the number of null values in the dataframe, we note that almost 23% of the entries are not assigned to a particular customer. Since these entries are not useful in this case, we remove all null values from the dataframe and confirm that no null values are left in the dataframe.

In [ ]:
online.dropna(axis = 0, subset = ['CustomerID'], inplace = True)
print('Dataframe dimensions:', online.shape)

# gives some infos on columns types and numer of null values
tab_info=pd.DataFrame(online.dtypes).T.rename(index={0:'column type'})
tab_info=tab_info.append(pd.DataFrame(online.isnull().sum()).T.rename(index={0:'number of null values'}))
tab_info=tab_info.append(pd.DataFrame(online.isnull().sum()/online.shape[0]*100).T.
                         rename(index={0:'null values (%)'}))
tab_info

Finally, we check for duplicate entries and remove them.

In [ ]:
print('Duplicate entries removed: {}'.format(online.duplicated().sum()))
online.drop_duplicates(inplace = True)

And we convert the data type of the CustomerID column to "object".

In [ ]:
online = online.astype({"CustomerID": int})
online = online.astype({"CustomerID": object})

In [ ]:
online.head()

In [ ]:
online.info()

And we see that after removing null values and duplicates we are left with almost 800K observations from the original dataframe to work with.

In [ ]:
len(online)

In [ ]:
online['InvoiceNo']

In [ ]:
online['order_canceled'] = online['InvoiceNo'].astype(str).apply(lambda x:int('C' in x))
display(online[:15])

n1 = online['order_canceled'].sum()
n2 = online.shape[0]
print('Number of orders canceled: {}/{} ({:.2f}%) '.format(n1, n2, n1/n2*100))

In [ ]:
online['QuantityCanceled'] = 0

entry_to_remove = [] ; doubtfull_entry = []

for index, col in  online.iterrows():
    if (col['Quantity'] > 0) or col['Description'] == 'Discount': continue        
    df_test = online[(online['CustomerID'] == col['CustomerID']) &
                         (online['StockCode']  == col['StockCode']) & 
                         (online['InvoiceDate'] < col['InvoiceDate']) & 
                         (online['Quantity']   > 0)].copy()
    #_________________________________
    # Cancelation WITHOUT counterpart
    if (df_test.shape[0] == 0): 
        doubtfull_entry.append(index)
    #________________________________
    # Cancelation WITH a counterpart
    elif (df_test.shape[0] == 1): 
        index_order = df_test.index[0]
        online.loc[index_order, 'QuantityCanceled'] = -col['Quantity']
        entry_to_remove.append(index)        
    #______________________________________________________________
    # Various counterparts exist in orders: we delete the last one
    elif (df_test.shape[0] > 1): 
        df_test.sort_index(axis=0 ,ascending=False, inplace = True)        
        for ind, val in df_test.iterrows():
            if val['Quantity'] < -col['Quantity']: continue
            online.loc[ind, 'QuantityCanceled'] = -col['Quantity']
            entry_to_remove.append(index) 
            break

In [ ]:
print("entries_to_remove: {}".format(len(entry_to_remove)))
print("doubtfull_entries: {}".format(len(doubtfull_entry)))

In [ ]:
online.drop(entry_to_remove, axis = 0, inplace = True)
online.drop(doubtfull_entry, axis = 0, inplace = True)
remaining_entries = online[(online['Quantity'] < 0) & (online['StockCode'] != 'D')]
print("number of cancelled entries that have not been deleted: {}".format(remaining_entries.shape[0]))
remaining_entries[:5]

In the final step of data preparation we remove the _StockCode_, _order_canceled_ and _QuantityCanceled_ since we don't need them anymore.

In [ ]:
online.drop(['StockCode', 'order_canceled', 'QuantityCanceled'], axis = 1, inplace = True)

## 2. Recency, Frequency, and Monetary Value analysis

### 2.1 Calculate RFM values

Next, we will work on behavioral customer segmentation based on three metrics:

- Recency(R)

- Frequency(F)

- MonetaryValue(M)

First, we create a TotalSum column in the dataset by multiplying Quantity and UnitPrice 

In [ ]:
# Create a TotalSum column in the dataset by multiplying Quantity and UnitPrice
online['TotalSum']= online['Quantity'] * online['UnitPrice']

Then, we create create a hypothetical snapshot_day variable as if we're doing analysis recently, that we can use to calculate recency.

In [ ]:
snapshot_date = max(online.InvoiceDate) + datetime.timedelta(days=1)

Then, we calculate RFM values

In [ ]:
# Calculate Recency, Frequency and Monetary value for each customer 
datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})

# Rename the columns 
datamart.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'TotalSum': 'MonetaryValue'}, inplace=True)

# Print top 5 rows
print(datamart.head())

In [ ]:
datamart.describe()

We have successfully built a dataset with recency, frequency, and monetary values.

### 2.2 Building RFM segments

#### 2.2.1 Calculate 3 groups for recency and frequency

We will now group the customers into three separate groups based on Recency, and Frequency.

We will use the result from this step in the next step, where we will group customers based on the MonetaryValue and finally calculate an RFM_Score.
Once completed, we print the results to the screen to make sure we have successfully created the quartile columns.

In [ ]:
# Create labels for Recency and Frequency
r_labels = range(3, 0, -1); f_labels = range(1, 4)

# Assign these labels to three equal percentile groups 
r_groups = pd.qcut(datamart['Recency'], q=3, labels=r_labels)

# Assign these labels to three equal percentile groups 
f_groups = pd.qcut(datamart['Frequency'], q=3, labels=f_labels)

# Create new columns R and F 
datamart = datamart.assign(R=r_groups.values, F=f_groups.values)

# Create labels for MonetaryValue
m_labels = range(1, 4)

# Assign these labels to three equal percentile groups 
m_groups = pd.qcut(datamart['MonetaryValue'], q=3, labels=m_labels)

# Create new column M
datamart = datamart.assign(M=m_groups.values)

datamart

In the next step we will finalize the RFM segmentation by assigning customers to three groups based on the MonetaryValue percentiles and then calculate an RFM_Score which is a sum of the R, F, and M values.

In [ ]:
# Calculate RFM_Score
datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)
print(datamart['RFM_Score'].head())

We have now built the critical components for RFM segmentation.

#### 2.2.2 Creating custom segments

Now we will  create a custom segmentation based on RFM_Score values. You will create a function to build segmentation and then assign it to each customer.

We create segments named Top, Middle, Low. If the RFM score is greater than or equal to 9, the level should be "Top". If it's between 6 and 9 it should be "Middle", and otherwise it should be "Low".

We will then apply the rfm_level function and store it as RFM_Level value.

In [ ]:
# Define rfm_level function
def rfm_level(df):
    if df['RFM_Score'] >= 9:
        return 'Top'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 9)):
        return 'Middle'
    else:
        return 'Low'

# Create a new variable RFM_Level
datamart['RFM_Level'] = datamart.apply(rfm_level, axis=1)

# Print the header with top 5 rows to the console
print(datamart.head(20))

We have successfully created a custom segment based on RFM Score.

#### 2.2.3 Analyzing custom segments

As a final step, we will analyze average values of Recency, Frequency and MonetaryValue for the custom segments we've created.

We calculate the averages for Recency, Frequency and MonetaryValue for each _RFM_Level_ segment.

As the last column, we return the size of each segment passing count to the MonetaryValue column next to the mean.
We then print the aggregated dataset.

In [ ]:
# Calculate average values for each RFM_Level, and return a size of each segment 
rfm_level_agg = datamart.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
  
  # Return the size of each segment
    'MonetaryValue': ['mean', 'count']
}).round(1)

# Print the aggregated dataset
print(rfm_level_agg)

## 3. Data pre-processing for clustering

### 3.1 Building RFM segments

After we have created segments, we want to make predictions. However, we first need to master practical data preparation methods to ensure our k-means clustering algorithm will uncover well-separated, sensible segments.

We will load the dataset with RFM values that we calculated previously as _datamart_rfm_ to explore the distributions of the RFM values.

In [ ]:
# Load the dataset with RFM values that we calculated previously as datamart_rfm
datamart_rfm = datamart[['Recency', 'Frequency', 'MonetaryValue']]
datamart_rfm = datamart_rfm[datamart_rfm['MonetaryValue'] > 0]
datamart_rfm

The following is the distribution of the raw data.

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(8, 10), dpi=80)

# Plot recency distribution
plt.subplot(3, 1, 1); sns.histplot(datamart_rfm['Recency'])

# Plot frequency distribution
plt.subplot(3, 1, 2); sns.histplot(datamart_rfm['Frequency'])

# Plot monetary value distribution
plt.subplot(3, 1, 3); sns.histplot(datamart_rfm['MonetaryValue'])

# Show the plot
plt.show()

We now have a good understanding about the distributions of RFM variables.

### 3.2 Pre-process RFM data

Since the variables are skewed and are on different scales, we will now un-skew and normalize them.

We apply log transformation to unskew the _datamart_rfm_ and store it as datamart_log. Then, we initialize a StandardScaler() instance as scaler and fit it on the datamart_log data. 

Next, we transform the data by scaling and centering it with scaler.

Then, we create a pandas DataFrame from _'datamart_normalized'_ by adding index and column names from datamart_rfm.

In [ ]:
datamart_rfm.describe() 

In [ ]:
# Unskew the data
datamart_log = np.log(datamart_rfm)

# Initialize a standard scaler and fit it
scaler = StandardScaler()
scaler.fit(datamart_log)

# Scale and center the data
datamart_normalized = scaler.transform(datamart_log)

# Create a pandas DataFrame
datamart_normalized_df = pd.DataFrame(data=datamart_normalized, index=datamart_rfm.index, columns=datamart_rfm.columns)

In [ ]:
datamart_normalized_df

In [ ]:
datamart_log

In [ ]:
datamart_rfm

We have unskewed and normalized the dataset!

### 3.3 Visualize the normalized variables

Now we will plot the normalized and unskewed variables to see the difference in the distribution as well as the range of the values. 

The following is the distribution of the normalized data.

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(8, 10), dpi=80)

# Plot recency distribution
plt.subplot(3, 1, 1); sns.histplot(datamart_normalized_df['Recency'])

# Plot frequency distribution
plt.subplot(3, 1, 2); sns.histplot(datamart_normalized_df['Frequency'])

# Plot monetary value distribution
plt.subplot(3, 1, 3); sns.histplot(datamart_normalized_df['MonetaryValue'])

# Show the plot
plt.show()

We can see how the skewness is managed after applying these transformations.

### 3.4 Correlation of the normalized dataset

In [ ]:
from string import ascii_letters

%matplotlib inline

sns.set_theme(style="white")

# Compute the correlation matrix
corr = datamart_normalized_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(14, 10))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

## 4. Customer Segmentation with K-means

### 4.1 Running k-means and assigning labels to raw data

In this section, we will use the data that we pre-processed in the previous section to identify customer clusters based on their recency, frequency, and monetary value.

In [ ]:
datamart_normalized_df

We see that:
- More recent customers (low recency) tend to buy more frequently.
- Monetary value tend to be higher for more recent customers (low recency).
- Customers that buys more frequently tend to generate a higher monetary value.

### 4.2 Choosing the number of clusters

We will now calculate the sum of squared errors for different number of clusters ranging from 1 to 20.

In [ ]:
# Fit KMeans and calculate SSE for each *k*
sse = {}

# Fit KMeans and calculate SSE for each k
for k in range(1, 11):
  
    # Initialize KMeans with k clusters and fit it on the normalized dataset
    kmeans = KMeans(n_clusters=k, random_state=1).fit(datamart_normalized)
    
    # Assign sum of squared distances to k element of dictionary
    sse[k] = kmeans.inertia_

Now we will plot the sum of squared errors for each value of k and identify if there is an elbow. This will guide us towards the recommended number of clusters to use.

In [ ]:
# Add the plot title "The Elbow Method"
plt.title('The Elbow Method')

# Add X-axis label "k"
plt.xlabel('k')

# Add Y-axis label "SSE"
plt.ylabel('SSE')

# Plot SSE values for each key in the dictionary
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

We can see the elbow is clearly around 3 clusters.

### 4.3 Building customer segments

We will now build a 3 clusters with k-means clustering. We have loaded the pre-processed RFM dataset as _datamart_normalized_df_.

In [ ]:
datamart_normalized= datamart_normalized_df.values

# Initialize KMeans
kmeans = KMeans(n_clusters=3, random_state=1) 

# Fit k-means clustering on the normalized data set
kmeans.fit(datamart_normalized)

# Extract cluster labels
cluster_labels = kmeans.labels_

In [ ]:
datamart_rfm.shape

We have now built a 3-cluster segmentation and extracted cluster labels.

We will now analyse the average RFM values of the three clusters that we have created in the previous step.

In [ ]:
# Create a DataFrame by adding a new cluster label column
datamart_rfm_k3 = datamart_rfm.assign(Cluster=cluster_labels)

# Group the data by cluster
grouped = datamart_rfm_k3.groupby(['Cluster'])

# Calculate average RFM values and segment sizes per cluster value
grouped.agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
  }).round(1)

In [ ]:
datamart_rfm_k3

We can now see the differences in RFM values of these segments.

### 4.4 Visualizing and evaluating the customer segments

Next, we visualize the customer segments, with the yellow cluster represting the "high" value customers.

In [ ]:
%matplotlib notebook

high = datamart_rfm_k3.query('Cluster == 0')
mid = datamart_rfm_k3.query('Cluster == 2')
low = datamart_rfm_k3.query('Cluster == 1')

from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

g1= (low['Frequency'].values, low['Recency'].values, low['MonetaryValue'].values)
g2 = (mid['Frequency'].values, mid['Recency'].values, mid['MonetaryValue'].values)
g3= (high['Frequency'].values, high['Recency'].values, high['MonetaryValue'].values)

data = [g1, g2, g3]
colors = ['#3ACF1B', '#FF7F00', '#3382FF']
groups = ['Low', 'Med', 'High']

for data, color, group in zip(data, colors, groups):
    x, y, z = data
    ax.scatter(x, y, z, alpha=0.5, c=color, label=group)

# Make legend
    ax.legend()
    ax.set_xlabel('Frequency')
    ax.set_ylabel('Recency')
    ax.set_zlabel('MonetaryValue')
    ax.set_title('Spatial Representation of Segments', loc='left')
    plt.show();

In [ ]:
# Calculate Silhoutte Score
score = silhouette_score(datamart_normalized, kmeans.labels_, metric='euclidean').round(2)
score

# 5. Finding growth opportunities

### 5.1 Top product keywords for low-value customers

Next, we get to know our customers, analyze the customer segments and understand what kind of customer groups and behaviors they represent.
We will focus on the low-value customer segments (cluster number 1), which is the customer segment that has the highest recenecy, lowest frequency and monetary value to improve our marketing strategy and offer them products that are related to their shopping interests.

In [ ]:
merged_df = pd.merge(online, datamart_rfm_k3, on='CustomerID')
merged_df = merged_df[['CustomerID', 'Country', 'Description', 'Cluster']].drop_duplicates()
merged_df.reset_index(drop=True, inplace = True)

low_value = merged_df[merged_df['Cluster'] == 1]
low_value.reset_index(drop=True, inplace = True)
low_value

As a first step, we extract from the Description variable the information that will prove useful. To do this, we use the following function:

In [ ]:
is_noun = lambda pos: pos[:2] == 'NN'

def keywords_inventory(dataframe, column = 'Description'):
    stemmer = nltk.stem.SnowballStemmer("english")
    keywords_roots  = dict()  # collect the words / root
    keywords_select = dict()  # association: root <-> keyword
    category_keys   = []
    count_keywords  = dict()
    icount = 0
    for s in dataframe[column]:
        if pd.isnull(s): continue
        lines = s.lower()
        tokenized = nltk.word_tokenize(lines)
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
        
        for t in nouns:
            t = t.lower() ; root = stemmer.stem(t)
            if root in keywords_roots:                
                keywords_roots[root].add(t)
                count_keywords[root] += 1                
            else:
                keywords_roots[root] = {t}
                count_keywords[root] = 1
    
    for s in keywords_roots.keys():
        if len(keywords_roots[s]) > 1:  
            min_length = 1000
            for k in keywords_roots[s]:
                if len(k) < min_length:
                    clef = k ; min_length = len(k)            
            category_keys.append(clef)
            keywords_select[s] = clef
        else:
            category_keys.append(list(keywords_roots[s])[0])
            keywords_select[s] = list(keywords_roots[s])[0]
                   
    print("Number of keywords in variable '{}': {}".format(column,len(category_keys)))
    return category_keys, keywords_roots, keywords_select, count_keywords

This function takes as input the dataframe and analyzes the content of the Description column by performing the following operations:

*   Extract the names (proper, common) appearing in the products description.
*   We extract the root of the word and aggregate the set of names associated with this particular root for each name.
*  Count the number of times each root appears in the dataframe.
*  When several words are listed for the same root, we consider that the keyword associated with this root is the shortest name (this systematically selects the singular when there are singular/plural variants).

The first step of the analysis is to retrieve the list of products from the "Description" variable:

In [ ]:
df_products = pd.DataFrame(low_value['Description'].unique()).rename(columns = {0:'Description'})

Once this list is created, we use the function that we previously defined in order to analyze the description of the various products:

In [ ]:
keywords, keywords_roots, keywords_select, count_keywords = keywords_inventory(df_products)

The execution of this function returns three variables:

*   Keywords: the list of extracted keywords
*   Keywords_roots: a dictionary where the keys are the keywords' roots and the values are the lists of words associated with those roots
*   Count_keywords: dictionary listing the number of times every word is used

At this point, we convert the count_keywords dictionary into a list, to sort the keywords according to their occurences:

In [ ]:
list_products = []
for k,v in count_keywords.items():
    list_products.append([keywords_select[k],v])
list_products.sort(key = lambda x:x[1], reverse = True)

Using list_products, we create a representation of the most common keywords:

In [ ]:
%matplotlib inline

list_words = sorted(list_products, key = lambda x:x[1], reverse = True)
#_______________________________
plt.rc('font', weight='normal')
fig, ax = plt.subplots(figsize=(7, 25))
y_axis = [i[1] for i in list_words[:125]]
x_axis = [k for k,i in enumerate(list_words[:125])]
x_label = [i[0] for i in list_words[:125]]
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 13)
plt.yticks(x_axis, x_label)
plt.xlabel("Number of occurences", fontsize = 18, labelpad = 10)
ax.barh(x_axis, y_axis, align = 'center')
ax = plt.gca()
ax.invert_yaxis()
#_______________________________________________________________________________________
plt.title("Words occurence",bbox={'facecolor':'k', 'pad':5}, color='w',fontsize = 25)
plt.show()

We see in the "Words occurance graph" that low-value customers' top 3 products contained the words "heart, "set", and "pink".
This information may be helpfull for improving marketing efforts directed to those customers.

### 5.2 Where are the low-value customers located?

In [ ]:
temp = low_value[['CustomerID', 'Description', 'Country']].groupby(['CustomerID', 'Description', 'Country']).count()
temp = temp.reset_index(drop = False)
countries = temp['Country'].value_counts()
print('Number of countries in the dataframe: {}'.format(len(countries)))

In [ ]:
cust_temp= low_value[['CustomerID', 'Country']]
cust_grouped = cust_temp.groupby('CustomerID')

cust_country = cust_grouped.apply(lambda x: x['Country'].unique())

cust_country_df = pd.DataFrame({'CustomerID':cust_country.index, 'Country':cust_country.values})

# Remove square brackets from the "Country" column
cust_country_df['Country'] = cust_country_df['Country'].str[0]

cust_country_df

In [ ]:
%matplotlib inline

cust_country_df['Country'].value_counts(normalize=True).plot.bar()

plt.show()

And we see that almost 100% of low-value customers are in the United Kingdom.